In [ ]:
import pandas as pd
import numpy as np
import random
import networkx as nx
from tqdm import tqdm
import re
import matplotlib.pyplot as plt
import ast

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [ ]:
# convert txt file to a dict
import ast
file = open("train_modified.txt", "r")

contents = file.read()
train_modified = ast.literal_eval(contents)
# train_mopdified will be the dictionary that represents the graph

file.close()

In [ ]:
# convert the sources and sinks in csv file into lists
train_set = pd.read_csv("train_set.csv", index_col = 0)


In [ ]:
# explore the mini datasets - check if all test_mini source and sink in train_mini_graph
source_num = 0
sink_num = 0
sink_sink = 0

for source in tqdm(train_set.train_source):
    if str(source) in train_modified.keys():
        source_num += 1
        
for sink in tqdm(train_set.train_sink):
    if str(sink) in train_modified.keys():
        sink_num += 1
        
#for sink in tqdm(train_set.train_sink):
##    for j in train_modified:
 #       if str(sink) in train_modified[str(j)]:
 #           sink_sink += 1
            
print(source_num, "source show up in train_mini_graph's source")
print(sink_num, "sink show up in train_mini_graph's source")
print(sink_sink, "test sink show up in train_mini_graph's sink")

In [ ]:
# people the source follow 
source_followee = {}  # len = 14610
for source in tqdm(train_set['train_source']):
    source = str(source)
    source_followee[source] = train_modified[source]
    
# number of people the source follow
each_followee = {}
for each in source_followee:
    each_followee[each] = len(source_followee[each])


# people follow the source
source_follower = {}                        
for source in tqdm(train_set['train_source']): 
    source_follower[str(source)] = []
    for key in train_modified.keys():
        if str(source) in train_modified[key]:
            source_follower[str(source)].append(key)



In [ ]:
# people the sink follow in the mini_train
sink_followee = {}  # len =  2016 
for sink in tqdm(train_set['train_sink']):
    sink = str(sink)
    if sink in train_modified.keys():
        sink_followee[sink] = train_modified[sink]

# number of people the sink follow
each_sink_followee = {}
for each in sink_followee:
    each_sink_followee[each] = len(sink_followee[each]) 
    
# people follow the sink in the mini_train
sink_follower = {} # len = 18342
for sink in tqdm(train_set['train_sink']): 
    sink_follower[str(sink)] = []
    for key in train_modified.keys():
        if str(sink) in train_modified[key]:
            sink_follower[str(sink)].append(key)

In [ ]:
len(train_set['train_sink'])

In [ ]:
# check if source_follower/sink_follower and source_followee/sink_followee have intersection parts
# # A-B 和 B-A 只算一次
# 得到people 同时follow和被follow的次数
intersect_source = {} # len = 14610
for key in tqdm(source_follower.keys()):
    intersect_num = 0
    for flr in source_follower[key]:
        if flr in source_followee[key]:
            intersect_num += 1
    intersect_source[key] = intersect_num


intersect_sink = {} # len = 2016
for key in tqdm(sink_followee.keys()):
    intersect_n = 0
    for flr in sink_followee[key]:
        if flr in sink_follower[key]:
            intersect_n += 1
    intersect_sink[key] = intersect_n    


In [ ]:
# source degree centrality
source_degree = {} # len = 14610
for source in train_set['train_source']:
    degree = each_followee[str(source)] + len(source_follower[str(source)]) - intersect_source[str(source)]
    source_degree[str(source)] = degree


In [ ]:
# sink degree centrality
sink_degree = {}

for sink in tqdm(train_set['train_sink']):
    if str(sink) in sink_followee.keys():
        degree = each_sink_followee[str(sink)] + len(sink_follower[str(sink)]) - intersect_sink[str(sink)]
        sink_degree[str(sink)] = degree
    elif str(sink) in sink_follower.keys():
        degree_new = len(sink_follower[str(sink)])
        sink_degree[str(sink)] = degree_new

In [ ]:
len(source_followee)

In [ ]:
a = [str(i) for i in train_set['train_sink']]
len(set(a))


In [ ]:
len(list(set([i for i in a if a.count(i)==1])))
# 613 > 1 的sink

In [ ]:
# sink array
sink_array = []
for i in tqdm(train_set['train_sink']): #19350
    sink_array.append(sink_degree[str(i)])


In [ ]:
# source array
source_array = []
for i in tqdm(train_set['train_source']): #19350
    source_array.append(source_degree[str(i)])
    

In [ ]:
train_set['source_degree'] = source_array
train_set['sink_degree'] = sink_array

In [ ]:
train_set.to_csv('degree_centrality.csv')
